## STATISTICS WORKSHOP  
  
__Victoria Lucia__  
__Gustavo A. Patino__  
  
Embark I  
March, 2019

__USING THE NOTEBOOK__  
The present notebook is composed of text and code cells. The former include the instructions for the activity and look just like regular text in a webpage. The code cells look like gray squares with empty square brackets to their left ([ ]). To run the code inside a code cells you'll need to click on it and then click "shift" + "enter", this will make the outcome of the come to appear underneath the cell.  
  
The next code cell will upload all the libraries and functions we'll need for the workshop. Please click "shift" + "enter" on it.

In [ ]:
# Loading Python libraries
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.multicomp as multi
from statsmodels.formula.api import ols
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.options.display.float_format = '{:.3f}'.format
np.set_printoptions(precision=3, suppress=True)


# Statistics functions
def parammct(data=None, independent=None, dependent=None):
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_numerical_categorical(data, independent, dependent):
        return
    
    parammct_df = pd.DataFrame()
    for value in pd.unique(data[independent]):
        mean = data[dependent][data[independent]==value].mean()
        stdev = data[dependent][data[independent]==value].std()
        n = data[dependent][data[independent]==value].count()
        sdemean = stdev/np.sqrt(n)
        ci = 1.96*sdemean
        lowerboundci = mean-ci
        upperboundci = mean+ci
        parammct_df[value] = pd.Series([mean, stdev, n, sdemean, lowerboundci, upperboundci], 
                                       index = ['Mean','SD','n','SEM','Lower bound CI', 'Upper bound CI'])
        
    return parammct_df


def non_parammct(data=None, independent=None, dependent=None):
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_numerical_categorical(data, independent, dependent):
        return
    
    non_parammct_df = pd.DataFrame()
    for value in pd.unique(data[independent]):
        median = data[dependent][data[independent]==value].median()
        minimum = data[dependent][data[independent]==value].quantile(0)
        q25 = data[dependent][data[independent]==value].quantile(0.25)
        q75 = data[dependent][data[independent]==value].quantile(0.75)
        maximum = data[dependent][data[independent]==value].quantile(1)
        non_parammct_df[value] = pd.Series([median, minimum, q25,q75, maximum], 
                                           index = ['Median', 'Minimum', 'Lower bound IQR', 'Upper bound IQR', 
                                                    'Maximum'])
        
    return non_parammct_df


def histograms(data=None, independent=None, dependent=None):
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_numerical_categorical(data, independent, dependent):
        return
    
    for value in pd.unique(data[independent]):
        sns.distplot(data[dependent][data[independent]==value], fit=stats.norm, kde=False)
        plt.title(dependent + ' by ' + independent + '(' + str(value).lower() + ')', 
                  fontweight='bold', fontsize=16)
        plt.ylabel('Frequency', fontsize=14)
        plt.xlabel(dependent, fontsize=14)
        plt.show()
    
    return 


def t_test(data=None, independent=None, dependent=None):
    
    pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
    independent_groups = pd.unique(data[independent])
    if len(independent_groups)>2:
        print('There are more than 2 groups in the independent variable')
        print('t-test is not the correct statistical test to run in that circumstance,')
        print('consider running an ANOVA')
        return
    
    mct = parammct(data=data, independent=independent, dependent=dependent)
    
    t_test_value, p_value = stats.ttest_ind(data[dependent][data[independent] == independent_groups[0]], 
                                            data[dependent][data[independent] == independent_groups[1]])
    
    difference_mean = np.abs(mct.loc['Mean'][0] - mct.loc['Mean'][1])
    pooled_sd = np.sqrt( ( ((mct.loc['n'][0]-1)*mct.loc['SD'][0]**2) + ((mct.loc['n'][1]-1)*mct.loc['SD'][1]**2) ) / 
                         (mct.loc['n'][0] + mct.loc['n'][1] - 2) )
    sedifference = pooled_sd * np.sqrt( (1/mct.loc['n'][0]) + (1/mct.loc['n'][1]) )
    difference_mean_ci1 = difference_mean + (t_test_value * sedifference)
    difference_mean_ci2 = difference_mean - (t_test_value * sedifference)
    if difference_mean_ci1>difference_mean_ci2:
        difference_mean_cilower = difference_mean_ci2
        difference_mean_ciupper = difference_mean_ci1
    else:
        difference_mean_cilower = difference_mean_ci1
        difference_mean_ciupper = difference_mean_ci2
    cohend = difference_mean / pooled_sd
    t_test_result= pd.Series ([difference_mean, sedifference, t_test_value, p_value, 
                               difference_mean_cilower, difference_mean_ciupper, cohend], 
                              index = ['Difference between means', 'SE difference', 't-test', 'p-value', 
                                       'Lower bound difference CI', 'Upper bound difference CI', 'Cohen\'s d'])
    
    return t_test_result


def anova(data=None, independent=None, dependent=None):
    
    pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_numerical_categorical(data, independent, dependent):
        return
    
    formula = dependent + ' ~ ' + independent
    model = ols(formula, data=data).fit()
    aov_table = sm.stats.anova_lm(model, typ=2)
    aov_table.rename(columns={'PR(>F)':'p'}, inplace=True)
    aov_table['F'] = pd.Series([aov_table['F'][0], ''], index = [independent, 'Residual'])
    aov_table['p'] = pd.Series([aov_table['p'][0], ''], index = [independent, 'Residual'])
    eta_sq = aov_table['sum_sq'][0]/(aov_table['sum_sq'][0]+aov_table['sum_sq'][1])
    aov_table['Eta squared'] = pd.Series([eta_sq, ''], index = [independent, 'Residual'])
    
    return aov_table


def tukey(data=None, independent=None, dependent=None):
    
    pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_numerical_categorical(data, independent, dependent):
        return
    
    test = multi.MultiComparison(data[dependent], data[independent])
    res = test.tukeyhsd()
    print(res.summary())
    
    return


def chi_square(data=None, variable1=None, variable2=None):
    
    pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
    
    variable1 = str(variable1)
    variable2 = str(variable2)
    if input_check_categorical_categorical(data, variable1, variable2):
        return
    
    values_var1=pd.unique(data[variable1])
    values_var2=pd.unique(data[variable2])
    
    problem_found=False
    for variable in [values_var1, values_var2]:
        if len(variable)<2:
            print(variable, 'has less than two categories. It has:', len(variable))
            problem_found=True
    if problem_found:
        return
    
    contingency_table = pd.crosstab(data[variable1], data[variable2])
    print('\033[1m' + 'Contingency Table' + '\033[0m')
    print(contingency_table, '\n\n')
    print('\033[1m' + 'Chi-square results' + '\033[0m')
    
    chi2_test=stats.chi2_contingency(contingency_table, correction=False)
    
    chi2_result= pd.Series ([chi2_test[0], chi2_test[1], chi2_test[2], chi2_test[3]], 
                            index = ['Chi-square value', 'p-value', 'Degrees of freedom', 'Expected frequencies'])
    
    return chi2_result


def logistic_reg(data=None, independent=None, dependent=None):
    
    pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)
    
    independent = str(independent)
    dependent = str(dependent)
    if input_check_categorical(data, independent, dependent):
        return
    
    if not len(pd.unique(data[dependent]))==2:
        print('Dependent variable must have two categories')
        print(dependent, 'variable has', len(pd.unique(data[dependent])), 'categories')
        return
    
    data['interceptant']=1
    independent=[independent, 'interceptant']
    logReg = sm.Logit(data[dependent], data[independent])
    regression = logReg.fit()
    print(regression.summary(), '\n')
    print('\033[1m' + 'Coefficients confidence intervals' + '\033[0m')
    print(regression.conf_int())
    
    predicted_values =regression.predict()
    plt.plot(data['age'], data['osas'], 'o', label='Actual values')
    plt.plot(data['age'], predicted_values, 'ok', label='Predicted probabilities')
    plt.xlabel('Age', fontsize=14)
    plt.ylabel('OSAS', fontsize=14)
    plt.ylim(-0.05, 1.05)
    plt.legend()
    plt.show()
    
    return


# Functions to validate statistical functions inputs
def input_check_numerical_categorical(data, independent, dependent):
    
    problem_found=check_input_dataframe(data)
    if check_variable_specified(independent):
        print ('An independent variable was not specified')
        problem_found=True
    if check_variable_specified(dependent):
        print ('A dependent variable was not specified')
        problem_found=True
    if problem_found:
        return problem_found
    
    if check_variables_are_columns(data, independent, dependent):
        return True
    
    if check_variable_types(data, dependent, ['int', 'float']):
        problem_found=True
    if check_variable_types(data, independent, ['bool', 'category']):
        problem_found=True
    
    return problem_found


def input_check_numerical_numerical(data, variable1, variable2):
    
    problem_found=check_input_dataframe(data)
    if check_variable_specified(variable1) or check_variable_specified(variable2):
        print ('Two variables must be specified')
        problem_found=True
    if problem_found:
        return problem_found
    
    if check_variables_are_columns(data, variable1, variable2):
        return True
    
    for variable in [variable1, variable2]:
        if check_variable_types(data, variable, ['int', 'float']):
            problem_found=True

    return problem_found


def input_check_categorical_categorical(data, variable1, variable2):
    
    problem_found=check_input_dataframe(data)
    if check_variable_specified(variable1) or check_variable_specified(variable2):
        print ('Two variables must be specified')
        problem_found=True
    if problem_found:
        return problem_found
    
    if check_variables_are_columns(data, variable1, variable2):
        return True
    
    for variable in [variable1, variable2]:
        if check_variable_types(data, variable, ['bool', 'category']):
            problem_found=True

    return problem_found


def input_check_categorical(data, independent, dependent):
    
    problem_found=check_input_dataframe(data)
    if check_variable_specified(independent):
        print ('An independent variable was not specified')
        problem_found=True
    if check_variable_specified(dependent):
        print ('A dependent variable was not specified')
        problem_found=True
    if problem_found:
        return problem_found
    
    if check_variables_are_columns(data, independent, dependent):
        return True
    
    if check_variable_types(data, dependent, ['bool', 'category']):
        problem_found=True

    return problem_found


# Functions to validate individual inputs
def check_input_dataframe(data):

    if not str(type(data))=='<class \'pandas.core.frame.DataFrame\'>':
        print (data, 'is not a DataFrame')
        return True 
    else:
        return False


def check_variable_specified(variable):

    if variable==None:
        return True
    else:
        return False


def check_variable_is_column(data, variable):

    if variable not in data.columns:
        print (variable, 'is not a column of', data, 'dataset')
        return True
    else:
        return False


def check_variables_are_columns(data, variable1, variable2):

    problem_found=False
    for variable in [variable1, variable2]:
        if check_variable_is_column(data, variable):
            problem_found=True
    return problem_found


def check_variable_types(data, variable, data_types):

    if data[variable].dtypes not in data_types:
        print (variable, 'is not of', data_types, 'type')
        return True
    else:
        return False

__LOADING THE DATABASE__  
In this exercise we will use a database of patients evaluated for obstructive sleep apnea syndrome (OSAS). Each patient filled out a survey where epidemiological characteristics and symptoms were recorded. The database will contain some of those characteristics along with whether they had OSAS or not, and its severity, based on a measure of how frequently the patient stops breathing through the nigh called the Apnea-Hypopnea Index (ahi).  
  
We will upload the data we'll work into memory from a CSV file in the website GitHub and put it in a variable called "data". Please execute the following code cells.

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/gapatino/stats-notebooks/master/stats_workshop_database.csv")

Then define some of the columns in the database as categorical variables

In [ ]:
data['gender']=data['gender'].astype('category')
data['osas_severity']=data['osas_severity'].astype('category')

Let's look at the data by displaying the first 10 rows of it

In [ ]:
data.head(10)

__APPLICATION EXERCISE__  
Below you will find questions about analyzing this data. After each question you will find a code cell and a text cell. Please enter the code for the appropriate statistical test in the code cell below it and run it, based on the output of the test answer the question in the text cell.  
If you need additional code cells you can add them by clicking on the button with the plus sign at the top of the page.

__Question 1__  
What is the type of each variable (column) in the dataset table?  
Hint: You don't need to run any functions to answer this

__Question 2__  
What is the mean and standard deviation of the age of male subjects?

__Question 3__  
Does the BMI values have a normal distribution across OSAS patients and controls?

__Question 4__  
What is the median and interquartile range of BMI among smokers?

__Question 5__  
What is the range of AHI among subjects that snore?

__Question 6__  
How many levels of OSAS severity are there and how many subjects are in each of them?

__Question 7__  
Is there a difference in the mean age of subjects with and without OSAS?

__Question 8__  
Is there a difference in the mean BMI of subjects across the severity levels of OSAS?

__Question 9__  
Is there a difference in the number of subjects with apnea between those with and without OSAS?

__Question 10__  
Can the age predict if a subject will have OSAS?

__Question 11__  
Did you find this session useful?

In [1]:
import ipywidgets as widgets
widgets.RadioButtons(
    options=['Yes', 'No'],
    description=' ',
    disabled=False
)

RadioButtons(description=' ', options=('Yes', 'No'), value='Yes')

__Question 12__  
Would you prefer to have future statistics sessions delivered as regular lectures or hands-on exercises like this one?

In [2]:
widgets.RadioButtons(
    options=['Yes', 'No'],
    description=' ',
    disabled=False
)

RadioButtons(description=' ', options=('Yes', 'No'), value='Yes')